## Preprocessing

In [1]:
user = 'X'
model_name = 'metal_prediction_CNN'

In [2]:
import sys
import json

print ("Initializing global variables...", end=' ')
sys.stdout.flush()

# Filepaths
output_file = './logs/results.txt'
hist_path = model_path = fig_path = './logs/'
dict_path = './dictionaries/'

print ("Done")
print ("  Filepath set to ./logs/")

##################################################

print ("Importing modules...", end=' ')
import metal_prediction_modules
print ("Done")

##################################################

print ("Reading data from disk...", end=' ')
sys.stdout.flush()

import numpy as np
import pandas as pd

df = pd.read_parquet('./datasets/Metal_all_20180601.parquet')
seqs = np.array(df.sequence)
target = np.array(df.ligandId)
cluster_numbers = np.array(df.clusterNumber90)

print (set(target))

label_dict ={}
for i, j in enumerate(set(df.ligandId)):
    label_dict[j] = i

for i in range(target.shape[0]):
    target[i] = [label_dict[target[i]]]
    

print ("Done")

##################################################

print ("Loading dictionaries...", end=' ')
sys.stdout.flush()

# FOFE
vocab_dic_fofe = {}
with open(dict_path + "vocab_dict_fofe", 'r') as fp:
        vocab_dic_fofe = json.load(fp)

print ("Done")

##################################################

print ("Performing cross validation split...", end=' ')
ratio = 0.9
split = int(ratio*len(seqs))
train_seqs, val_seqs = seqs[:split], seqs[split:]
train_label, val_label = target[:split], target[split:]
print ("Done")
print ("  Ratio :", ratio)
print ("  Train_range :", 0, "-", split-1)
print ("  Val_range :", split, "-", len(seqs)-1)

Initializing global variables... Done
  Filepath set to ./logs/
Importing modules... 

Using TensorFlow backend.


Done
Reading data from disk... {'CA', 'ZN', 'NI', 'CO', 'FE', 'CU', 'MG', 'MN'}
Done
Loading dictionaries... Done
Performing cross validation split... Done
  Ratio : 0.9
  Train_range : 0 - 52385
  Val_range : 52386 - 58206


In [3]:
df.groupby('ligandId').count()

,structureChainId,fingerprint,groupNumber,sequence,interactingChains,clusterNumber30,clusterNumber40,clusterNumber50,clusterNumber70,clusterNumber90,clusterNumber95,clusterNumber100
ligandId,,,,,,,,,,,,
CA,17606,17606,17606,17606,17606,17606,17606,17606,17606,17606,17606,17606
CO,759,759,759,759,759,759,759,759,759,759,759,759
CU,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143,2143
FE,2989,2989,2989,2989,2989,2989,2989,2989,2989,2989,2989,2989
MG,8336,8336,8336,8336,8336,8336,8336,8336,8336,8336,8336,8336
MN,4688,4688,4688,4688,4688,4688,4688,4688,4688,4688,4688,4688
NI,1073,1073,1073,1073,1073,1073,1073,1073,1073,1073,1073,1073
ZN,20613,20613,20613,20613,20613,20613,20613,20613,20613,20613,20613,20613


## Data Generator

- <font color=blue>FOFE Encoding</font>

In [4]:
train_args = {'sequences': train_seqs,
              'labels': train_label,
              'translator': vocab_dic_fofe}
val_args = {'sequences': val_seqs,
            'labels': val_label,
            'translator': vocab_dic_fofe}
common_args = {'batch_size': 100,
               'input_shape': (800,),
               'label_shape': (8, ),
               'shuffle': True}

train_gen = metal_prediction_modules.FOFEGenerator(**train_args, **common_args)
val_gen = metal_prediction_modules.FOFEGenerator(**val_args, **common_args)

## Model
- <font color=blue>CNN</font>

In [5]:
# ProtVec:100, One-hot:20, blosum62:20, property:7
dimension = 800
cutoff = 8

import tensorflow as tf
import time
import matplotlib.pyplot as plt
% matplotlib inline
np.random.seed(2017) 
from keras.models import Sequential, Model
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D, AveragePooling2D
from keras.layers import Activation, Flatten, Dense, Dropout, Reshape, Embedding, Input
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.optimizers import SGD
import numpy as np
import keras
from keras.models import Model, load_model
from keras.optimizers import Adam, SGD, RMSprop
# Visualization
from keras.utils import plot_model

input_shape = (dimension,)

input_0 = Input(shape=input_shape, dtype='float32')
input_0_reshape = Reshape((1,dimension,1), input_shape=(dimension,))(input_0)
conv2d_3 = Convolution2D(2, 1, 3, border_mode='same')(input_0_reshape)
conv2d_5 = Convolution2D(2, 1, 5, border_mode='same')(input_0_reshape)
conv2d_7 = Convolution2D(2, 1, 7, border_mode='same')(input_0_reshape)

x = keras.layers.concatenate([conv2d_3,conv2d_5,conv2d_7])
x = Activation('relu')(x)
x = Flatten()(x)
x = Dense(cutoff, activation='relu')(x)
output_0 = Dense(cutoff, activation='softmax')(x)
#output_0_reshape = Reshape((cutoff,1), input_shape=(cutoff,))(output_0)

#model = Model(inputs=input_0, outputs=output_0_reshape)
model = Model(inputs=input_0, outputs=output_0)                              
# end of the MODEL

sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

# model.summary()

C:\Users\Tian\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, (1, 3), padding="same")`
C:\Users\Tian\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, (1, 5), padding="same")`
C:\Users\Tian\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, (1, 7), padding="same")`


In [6]:
model_args = {'model': model, 
              'generators': [train_gen, val_gen], 
              'callbacks': [], 
              'post_train_args': {'user': user, 
                                  'model': model_name, 
                                  'result': output_file, 
                                  'fig_path': fig_path}}

trainer = metal_prediction_modules.Trainer(**model_args)

Assigning validation generator... Done
Matching input shape... Done
Matching output shape... Done
Trainer initialized.


In [9]:
import warnings; 
warnings.simplefilter('ignore')
trainer.start(epoch=5)

Epoch 1/5
523/523 [==============================] - 78s 150ms/step - loss: 0.3396 - acc: 0.8772: 2:00 - loss: 0.2993 - acc: - ETA: 1:37 - loss: 0.3061 - acc: 0.8 - ETA: 1:26 - loss: 0.3147 - acc: 0. - ETA: 1:26 - loss: 0.3139 - ac - ETA: 1:19 - loss: 0.3111 - acc: - ETA: 1:18 - loss: 0.3045 - acc - ETA: 1:16 - loss: 0.3039 - ac - ETA: 1:17 - loss: 0.2980 - acc: 0 - ETA: 1:18 - loss: 0.3019 - acc:  - ETA: 1:15 - loss: 0.30 - ETA: 1:09 - loss: - ETA: 1:05 - loss: 0.3004 - a - ETA: 58s - loss: 0.3123 - acc - ETA: 56s - loss: 0.3139 - - ETA: 55s - loss: 0.31 - ETA: 53s - loss: 0.31 - ETA: 52s - loss: 0.3184 - acc:  - E - ETA: 49s - loss: 0.32 - ETA: 47s - loss: 0.3230 - acc:  - ETA: 46s - loss: 0.3230 - - ETA: 46s -  - ETA: 44s - loss: 0.3275 - acc:  - ETA: 43s - loss - ETA: 38s - loss:  - ETA: 37s -  - ETA: 28s - loss: 0.3315 - acc:  - ETA: 28s - loss: 0.3315 - a - ETA: 27s - loss: 0.3312 - ETA: 26s - loss: 0.3319 - acc: 0.88 - ETA: 26s - loss: 0.3320 - ETA: 22s - loss: 0.33 - ETA: 20s -

In [10]:
# serialize model to JSON
model_json = model.to_json()
with open("./models/metal_predict.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("./models/metal_predict.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
# later...
from keras.models import model_from_json
# load json and create model
json_file = open('type.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("type.h5")
print("Loaded model from disk")

In [ ]:
train_args = {'sequences': seqs,
              'labels': target,
              'translator': vocab_dic_fofe}
common_args = {'batch_size': 1,
               'input_shape': (800,),
               'label_shape': (13, ),
               'shuffle': False}
train_gen = modules_type.FOFEGenerator(**train_args, **common_args)

In [ ]:
metal_predictions = []
Y = []
for i in range(len(train_gen)):
    x,y = train_gen[i]
    metal_predictions.append(model.predict(x))
    Y.append(y)
    

In [ ]:
inv_label_dict = {v: k for k, v in label_dict.items()}
l1 = []
l2 = []
for i,j in enumerate(Y):
    max_i = max(j[0])
    x = [a for a, b in enumerate(Y[i][0]) if b == max_i]
    l2.append(inv_label_dict[x[0]])
    
for i,j in enumerate(metal_predictions):
    max_i = max(j[0])
    x = [a for a, b in enumerate(metal_predictions[i][0]) if b == max_i]
    l1.append(inv_label_dict[x[0]])

In [ ]:
c = 0
for i in range(len(l1)):
    if l1[i] != l2[i]:
        c+=1
print ((len(l1)-c) / len(l1))

In [ ]:
df['metalPrediction'] = np.array(l1, dtype='O')

In [ ]:
df['metalPrediction'].dtype

In [ ]:
df

In [ ]:
df.to_json('Metal_all_20180601_predicted.parquet', orient='index')

In [ ]:
df2 = pd.read_json('Metal_all_20180601_predicted.parquet', orient='index')

In [ ]:
df2

In [ ]:
print (l1[:10])

In [ ]:
print (l2[:10])

In [ ]:
# Remove seqs containing 'U' and 'X'

duplicate_dict = {}
rows_to_delete = []
for i in range(seqs.shape[0]):
    if 'X' in seqs[i] \
    or 'U' in seqs[i] \
    or '3CO' in target[i]\
    or '3NI' in target[i] \
    or 'FE2'in target[i] \
    or 'CU1'in target[i]\
    or 'MN3' in target[i] \
    or np.isnan(cluster_numbers[i]):
        rows_to_delete.append(i)
        print (i, end=',')
    elif seqs[i] not in duplicate_dict.keys():
        duplicate_dict[seqs[i]] = target[i]
    else:
        if target[i] != duplicate_dict[seqs[i]]:
            rows_to_delete.append(i)
            print (i, end=',')
    
# df = df.drop(df.index[rows_to_delete])
# df.to_parquet('Metal_all_20180601.parquet')
seqs = np.delete(seqs, rows_to_delete, 0)
target = np.delete(target, rows_to_delete)
cluster_numbers = np.delete(cluster_numbers, rows_to_delete)